In [2]:
!pip install entrypoints

You should consider upgrading via the '/home/reibomaru/.anyenv/envs/pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
!pip install huggingface_hub > /dev/null

You should consider upgrading via the '/home/reibomaru/.anyenv/envs/pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [4]:
!pip install langchain 

You should consider upgrading via the '/home/reibomaru/.anyenv/envs/pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [5]:
!pip install openai

You should consider upgrading via the '/home/reibomaru/.anyenv/envs/pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [6]:
!pip install chromadb

You should consider upgrading via the '/home/reibomaru/.anyenv/envs/pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [7]:
!pip install tiktoken

You should consider upgrading via the '/home/reibomaru/.anyenv/envs/pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [8]:
!pip list

Package                  Version
------------------------ ----------
aiohttp                  3.8.4
aiosignal                1.3.1
anyio                    3.5.0
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
asttokens                2.0.5
async-timeout            4.0.2
attrs                    21.4.0
autopep8                 1.6.0
Babel                    2.9.1
backcall                 0.2.0
backoff                  2.2.1
beautifulsoup4           4.10.0
bleach                   4.1.0
certifi                  2022.12.7
cffi                     1.15.0
charset-normalizer       3.1.0
chromadb                 0.3.21
click                    8.1.3
clickhouse-connect       0.5.22
cmake                    3.26.3
dataclasses-json         0.5.7
debugpy                  1.6.0
decorator                5.1.1
defusedxml               0.7.1
duckdb                   0.7.1
entrypoints              0.4
executing                0.8.3
fastapi                  0.95.1
fastjsonschema       

In [9]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

 ········


In [10]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [11]:
from langchain import HuggingFaceHub

repo_id = "stabilityai/stablelm-tuned-alpha-3b" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0, "max_length":64})

/home/reibomaru/.anyenv/envs/pyenv/versions/3.8.16/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Who is the current Prime Minister of Japan?"

print(llm_chain.run(question))

Who is the current Prime Minister of Japan?As of August 2021, the current Prime Minister of Japan is Naoto Kan.


In [13]:
from getpass import getpass

OPENAI_API_KEY = getpass()

 ········


In [14]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [35]:
from langchain.document_loaders import TextLoader
loader = TextLoader('./bleach_summary.txt', encoding="utf-8")

In [20]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

Using embedded DuckDB without persistence: data will be transient


In [21]:
index.query("What did the president say about Ketanji Brown Jackson")

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He also said that she is a consensus builder and has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."

In [22]:
query = "What did the president say about Ketanji Brown Jackson"
index.query_with_sources(query)

{'question': 'What did the president say about Ketanji Brown Jackson',
 'answer': " The president said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, one of the nation's top legal minds, who will continue Justice Breyer's legacy of excellence, and that she has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.\n",
 'sources': './state_of_the_union.txt'}

In [36]:
from langchain.text_splitter import CharacterTextSplitter

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [37]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [38]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings)

Using embedded DuckDB without persistence: data will be transient


In [39]:
retriever = db.as_retriever()

In [40]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [43]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="map_reduce", retriever=retriever)

In [45]:
query = "Bleachの登場人物であるチャドはどのような活躍をしましたか？"
qa.run(query)

'チャドは、藍染が織姫を拐った理由である護廷十三番隊の戦力の分散を阻止するため、一護と共に空座町をレプリカとして襲撃した際の助けとなり、ユーハバッハに能力を一瞬無にする「銀の鏃」を打ち込むなどの活躍をした。'

In [46]:
query = "Bleachの登場人物である雨竜はどのような活躍をしましたか？"
qa.run(query)

' 雨竜は、藍染が織姫を拐われた事件において、虚園へと織姫救出に向かい、ユーハバッハの攻撃を仕掛ける隙を見逃さず攻撃し、ルキアの奪還に向かう活躍をしました。'

In [47]:
query = "どうして黒崎一護は死神代行になったのですか？"
qa.run(query)

" I don't know."

In [48]:
query = "黒崎一護のライバルは誰ですか？"
qa.run(query)

' 黒崎一護のライバルは雨竜、織姫、チャド、夜一、ユーハバッハ、藍染の6人です。'